# Job Market Assistant


## Initialization


In [2]:
import time

import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown
from linkedin_jobs_scraper.query import Query, QueryOptions

from core.job_market_assistant import JobMarketAssistant
from core.models import ChatGPT, Gemini, Llama
import vertexai
from google.oauth2.service_account import Credentials

# ChatGPT, Llama
load_dotenv()

# Gemini
credentials = Credentials.from_service_account_file("gemini.json")
vertexai.init(
    project="linkedln-advisor", location="europe-central2", credentials=credentials
)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Framework

Our class-based structure allows to encapsulate reused code and perform experiments easily by adding new models and altering the prompts.


All the models have to implement _complete_ method. With parameter _debug_ set to _True_ all the querying history is visible.


In [5]:
debug_chat_gpt = ChatGPT(True)
completion = debug_chat_gpt.complete("Hello Chat!")

>>> ChatGPT: Hello Chat!
ChatGPT >>>: How is everyone?

I am a language model AI, so I don't have feelings like humans. But I am always happy to chat and help out with any questions you may have! Is there anything specific you would like to talk about?


In [4]:
debug_gemini = Gemini(True)
completion = debug_gemini.complete("Hello Gemini!")

>>> Gemini: Hello Gemini!
Gemini >>>: Hello! It's nice to meet you. What would you like to talk about today?


In [5]:
debug_llama = Llama(True)
completion = debug_llama.complete("Hello Llama!")

>>> Llama: Hello Llama!
Llama >>>: Hello there! It's nice to meet you! Is there something I can help you with or would you like to chat?


In [8]:
chat_gpt = ChatGPT()
gemini = Gemini()
llama = Llama()

The system uses 4 different prompts each serving unique purpose and having a distinct set of available parameters:

| Method             | Parameter             | Purpose                                                                      | Parameters                                 |
| ------------------ | --------------------- | ---------------------------------------------------------------------------- | ------------------------------------------ |
| get_cover_letter   | cover_letter_prompt   | Generate a cover letter based on the job posting and CV.                     | job_title, company, job_desc, location, cv |
| get_missing_skills | job_title_prompt      | Generate the desired _job_title_ from the CV unless provided explicitely     | cv                                         |
| get_missing_skills | key_points_prompt     | Summarize requirements for a position before feeding them to the next prompt | job_title, company, job_desc, location     |
| get_missing_skills | missing_skills_prompt | Identify skill gaps for the job title and return them as a list              | key_points, cv                             |


In [9]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

## Data sources


The job postings may be provided directly or scraped from LinkedIn using one of the helper methods.


In [4]:
file_job_postings = pd.read_feather("data/job_postings.feather")
file_job_postings

,job_title,company,job_desc,location
39,Applied Data Scientist,dunnhumby,dunnhumby is the global leader in Customer Dat...,"London, England, United Kingdom"
47,Data Scientist,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
267,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
147,"Software Development Engineer, Caching",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
265,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom"
...,...,...,...,...
183,Sales Business Analyst,Aer Lingus,Your role\n\nAre you ready to make a significa...,"Dublin, County Dublin, Ireland"
20,"Software Development Engineer, AWS Security Se...",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
81,Application & Service Engineer,Altus Group Ltd,I am pleased to announce that we are looking f...,"Cork, County Cork, Ireland"
327,Graduate Data Scientist,Worldpanel by Kantar,"We’re the world’s leading data, insights, and ...","Greater London, England, United Kingdom"


In [5]:
url_job_postings = JobMarketAssistant.scrap_urls(
    (
        "https://www.linkedin.com/jobs/view/3937215366/?alternateChannel=search&refId=pyO8PX6NFy5hyGNWQ5uRxQ%3D%3D&trackingId=aNQSGBRWWobujlSYEpbvaQ%3D%3D",
        "https://www.linkedin.com/jobs/view/3931218091/?alternateChannel=search&refId=%2BwGMIZDUEBj8w%2BcBBmKX0g%3D%3D&trackingId=MKGRxpSmYvT7yYfc0vnezg%3D%3D",
        "https://www.linkedin.com/jobs/view/3939354205/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Ryft9wMmyemu5LZP3YMVkQ%3D%3D&trk=d_flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3933974972/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Nf75Lj8H45f%2F3Wn0YZWGow%3D%3D&trk=d_flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3706100131/?eBP=CwEAAAGP2nLSS1cVH9CpQXLJCQYoBELJe_DMnb_FF0wnPFmYKZ8RevBvThpD_ekhE_mtBIX8fy5hK5ayxs7uQI8M551BR4OaZ0Vk6wIa_yi-E6P_orjzE4PKlU6nMCHwWnQoskuS4a0jCI8057qk3wYLKiI8pZ_mITVGGc_tOrpA5WVKMj_Zeoa44P2nxT1vJ1KUFmyeIiSBOAnIdhZu4fTCqmELBiPkEAD5eKftXGAuDr3pbYOWdt3TZ9lCMf-xY0rQGFaYih57c03_fADVuhjZygq2VkyoJpNIwGj_5INh7nT_fjMLCs_1POakqPL3o3-H7t2RAdFyjMtjYZ0D3_2500LfftGIcnS0Rrjl9Xss1xgu1HcRoGWs-Yx0hkoolePWe-KCNgbczGhVVMQkW3TEwVYwNmq-5ucGIuEvbNhqYo9H2th1MA7NLumAVxtEIWH6IK8TLe7JiAhf8VBblyQm2RgHaZTO&refId=ejemPX0qrd8qnaCrjBlNWw%3D%3D&trackingId=nRFdwwj3o5%2B%2B7KFMCCFo1w%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3926449844/?eBP=CwEAAAGP2nOgU8_sfnyPI_cLUth9VbgP3dIs3RClQtIwUPjlvFsnHuQAiIaEqWo-DXN0PMr8OSoQIj3obkACHeuwBugF67GnD_Z_IL8WkR2I8yAYJM3taMePkxBmPwI2zsd6facjFtS8XDaha22A0tXYebVPQscG8pvSd5vZSzyVY3r4GN9iBpOrZsgnD6Ql3f-o0NpW4H_Ea-GRZoVD5abwhA_FkiQjPW1_JL_b-MXYWhVlPXImXCYdh2s86zuIfyhmxYCto6nfZoLqgBGfNGa_g5NXMP7cF4tj57CaZxE2Bm22qe_vCdTfkUoiwWNVUa3rSEEtzB49dimLquOca5B2-Sa8IsYSJFTcpWw4GOInhexoSnI-Uawuu1AH_BBLQ-NqGXSAmpoIpbQKtCSID8PnSgUTJx_k4htjYF7FrG7ciSLAhVULZSLkCQ-UNYj9ubHhOzjuGpE1zvNnNKflzOLkmpXUiMg5USw&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=D9AGbek05Cnb7pxMYxiqpg%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3904363506/?eBP=CwEAAAGP2nOgU4zBnhBLcSx9mV0FYGVGgNyQ1sxXETXxVxPVFk53FLqXR9gH9f4U2Nqac6fmAWSBzAAnCbbBO-YY8F0YdhOwtuIQY01HY_W4sWI49I8PJP6pBSKCHPSDOj4lmCGZlDuy8LguqiBqKKAvNjUMPRw6yWT83z-Xo4YwNA62ju020hTZ-YFlK-WATuOZGhVoK0iwbySo89vy-cf2j5ZP7vpLrArpWUtzjNrGCItD_qOoCubYiF_93A_axtk8RkbmwmTIxL7AOl5Ql_veoGeNXdfdiw5wDLiGZBwp1lxvOIFiC_X7655cFQqm5jhB8p7jg8bRSYKSwbawikJntoTMDxs6TpvqjYHnbsT9xKaaPOpgDes3A4U3JONRtNa5S9y_-C3v9QWMC1DwwuPAqcETf7Llf47r4Npo-dxM17YgHj3FQicUzkI8wYSKpmzU0uK44dASHScYXwN5TWVX3pZxqEprZNY&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=vI35XF233bfRnGy6BAE%2FSQ%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3910213820/?eBP=CwEAAAGP2nOgU4o3HMYGtDJlo8CcLUUFwd-jc9D9_hbJ5mLrtYZcutRbb5mRXvtMJ1Ig92XZ0FOxY5SkMcrAfg3VK1tY1JObDFpgTe_y9Y7iNKbX94lMzuYnYBJn21Eeymf77lmdICyU6QH-yPQbdcf33E72AC_xt7EBxnpOv9M0sGXqJc11puvSVQGbXpW5W5Sbbsv6QTzs9FxlovRKu6pEPDCCVzjrHrC9d_gOylePjmUOnabDFkppFa3gbB8zh_il2KSTYB_lhCtl1Kxi-5y0NcsECFB5OR5KFUr0npWo7btVAewY5QBeyUFcbcX2O9pXjVdImXT40xQBaQbIUbZXEZeSEaFroWyIc-kB8FzaKMlLmIK5ezXmgfdGx_f10lWV_9xSFVlvaWI1MeG5adcsO0fTM2fY0ucITXYFS2qbispOnegSv3cS_p4WqGTNkLxg_Nxjr8ppKq8HMMZud8dppz5Izp2lOyg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=1OHZffvErAg6ayGGSMaGSQ%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3934612685/?eBP=CwEAAAGP2nOgUxurolZNvR-9t8O15gg8Hg73vyxdiM7INtIvQbruj1Lad7exeumIa4mrXe73RR3QJ3t-RkVakkFZqmPBu1QNpw4oyXWLO0A0d7RG_uAQJ0YJCdq34tT2c50_4HPgem5RvZ0Q_KcjsPUZuVDkiy0i5xxjusB-K9fzpJ1kHeoLJVsY4-Hdgjrd8hdvpQ6XwtPErHWtPsHmaBPgMvJfGw22Nu_SW6Or4WeOmLmaBroXMpeJGuVz7UqETkN7ro6h77j_Rsi25h8wYeL7S-NY6zlt5HCltq-cIXuFW_kpQ_FXPqHvnKuXkKpKchCBqgCSa3AUTCEaNF60I50pHS9xNHnhro0RaMODYkEAB1YhKBDLW_JY_qWIORfrs48oEWG5HCt_yCwEQCzO6ydupfJkjLOgCjEpOc3wFSMK6knh7yySP4zbSOArdmbPkMsaTTvgFHCG3loj9FRN49Dbrl5RFd1C3rg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=xDb049gA1gGIWmWSmZ6jfg%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3903916300/?eBP=CwEAAAGP2nWWOL08uhH7U-M5C0J0CphPf6gFN4SMMBAXxYf-YdOwMoRtXj0UBwSmzeTRcrgSE8fpf7yZ0xy0_rHcDoJxAtbx85i8FJ3WTzcEZNK18GYnCtNn8U_b7hhNTFgf3gJzXUEOtrn8L_uuthw7xG5LbY9Oro9myJvJKd2bZ84weNcA4EgTLVJ45AFVin1GecDXOkbJgXKf6pvqpQbBy_Fe8gxbM_A3ZJT6JrXKURhLempoczh7uTTM25y9ybmBJqnm7YjLTGRMy7DwoNIY6oZnVHWY_i04y5MPCDcGOVhn9bALuQ6s1XKZ6AsjEKbT3ytRKd2Q0LZnr7RSkfbk4LNYPtjKgQsVclC8Gao4t6-R76JebQn3GH65RNJr3zxogrDJk4OglgcaeasAou45tdfpBEj4l30DyYWUXcYqxNfsQxDlPaLGqEg30XR9bwKc8Kr57RFZ_16v5SccL6vYQoPO2g&refId=eGb0QUUekOjgqRmAiPqWmQ%3D%3D&trackingId=WfffkrTIdiq6gzEV2lzO5w%3D%3D&trk=flagship3_search_srp_jobs",
    )
)
url_job_postings

,job_title,company,job_desc,location
0,Machine Learning Engineer,Ampstek,"Hi Professionals,Greetings From Ampstek!!Hope ...",Poland
1,Machine Learning Consultant,Antdata,Antdata is a data analytics company that focus...,"Poznań, Wielkopolskie, Poland"
2,Senior Data Scientist with Python and ML,Luxoft Poland,Project descriptionSuccessful candidate will b...,Poland
3,Senior Artificial Intelligence Engineer,Tangent,"About TangentAt Tangent, we partner with large...","Poznań, Wielkopolskie, Poland"
4,Machine Learning Researcher,RTB House,RTB House owes its market position to an advan...,"Warsaw, Mazowieckie, Poland"
5,AI Engineer,Opera,"Location: Wrocław, PolandAbout the job We are ...","Wrocław, Dolnośląskie, Poland"
6,AI Developer with Python,Rockwell Automation,Rockwell Automation is a global technology lea...,"Katowice, Śląskie, Poland"
7,Machine Learning Engineer,Devoteam G Cloud,"Company DescriptionDevoteam, a group of 8000+ ...","Warsaw, Mazowieckie, Poland"
8,Generative AI Engineer,Seargin,As a dynamic multinational tech company operat...,Poland
9,AIMl GenAI Manager,EY,AIML GenAI ManagerLet us introduce you the job...,"Wrocław, Dolnośląskie, Poland"


In [6]:
query_job_postings = JobMarketAssistant.scrap_query(
    (Query("ML Engineer", options=QueryOptions(limit=5, locations=("Poland"))))
)
query_job_postings

CVs have to be provided as plain text. The desired *job_title* will be generated from the CV if not provided explicitely to the *get_missing_skills* method.

In [11]:
cv_1 = None
with open("data/cv_1.txt", "r") as file:
    cv_1 = file.read()

cv_2 = None
with open("data/cv_3.txt", "r") as file:
    cv_2 = file.read()

cv_3 = None
with open("data/cv_3.txt", "r") as file:
    cv_3 = file.read()

## Exemplary usage


In [7]:
cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

### Cover letter for Applied Data Scientist at dunnhumby:
[Your Name]
[Address]
[City, State ZIP Code]
[Email Address]
[Today’s Date]

[Employer’s Name]
[Company Name]
[Address]
[City, State ZIP Code]

Dear Hiring Manager,

I am excited to apply for the Applied Data Scientist position at dunnhumby, as advertised on [where you found the job posting]. With my educational background in Artificial Intelligence and Finance, combined with my experience as a C++ Developer and Cook, I believe I am a strong candidate for this role.

I am currently in my final year of Artificial Intelligence and have taken various courses such as Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining. These courses have equipped me with the necessary skills to become an Applied Data Scientist. Additionally, I am a member of GHOST, an academic circle focused on AI/ML, and I am currently part of the section organizing the GHOST conference about ML.

In my current role as a C++ Developer at CAD Projekt, I have gained experience in programming languages such as Python, C++, SQL, R, and Java, which are required for this position. I have also developed strong analytical techniques and a passion for connecting my work directly to the customer experience, making a real and tangible impact. My logical thinking and problem-solving skills, along with excellent communication skills, have enabled me to successfully work on complex applications and components in a team environment.

One of the things that caught my attention in the job posting was the company's focus on diversity and inclusion. I strongly believe in creating an inclusive culture where everyone has equal opportunities to thrive, and I am eager to contribute to this at dunnhumby.

As a person who values a work-life balance, I appreciate the company's approach to flexible working hours and thoughtful perks such as flexible working hours and your birthday off. I feel that this will create an environment where employees can perform their best and contribute to the company's success.

I am excited about the opportunity to work with cutting-edge technology at dunnhumby, and I am confident that my skills and experience make me a strong fit for this role. I am eager to learn and contribute to the company's mission of enabling businesses to grow and reimagine themselves by becoming advocates and champions for their Customers.

Thank you for considering my application. I have attached my resume for your review, and I look forward to the opportunity to discuss further my qualifications and how I can contribute to dunnhumby. Please

In [8]:
missing_skills = job_market_assistant.get_missing_skills(url_job_postings, cv_2)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title Computer Engineer and Data Science Student (derived from the CV)

,job_title,company,job_desc,location,score
0,Graduate Data Engineer,Oho Group Ltd,Graduate Data Engineer - Fast Growing FinTech\...,"London Area, United Kingdom",0.728353
1,"Data Scientist, People & Culture",Lifelancer,We are ideally looking for a candidate in the ...,"London, England, United Kingdom",0.717228
2,Data Scientist,Tilt,"Mission\n\nAt TILT, we’re pioneering the futur...","London, England, United Kingdom",0.712310
3,Data Engineer,ADLIB Recruitment | B Corp™,"Be part of a unique team, bridging commercial ...","Bristol, England, United Kingdom",0.712009
4,Data Engineer,Adria Solutions Ltd,We are seeking a Data Engineer to join our fri...,"Manchester, England, United Kingdom",0.711317
5,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.709216
6,Graduate Machine Learning Engineer,Mercedes-AMG PETRONAS Formula One Team,"Brackley, UK\n\nWe have an exciting opening fo...","Brackley, England, United Kingdom",0.702707
7,Analytics Engineer,Mars,Job Description\n\nThe Analytics Engineer at M...,"London, England, United Kingdom",0.702397
8,Data Scientist ( Hybrid),myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland",0.700871
9,Data Engineer,ecocareers,DATA ENGINEER LONDON HYBRID - 3 DAYS IN THE OF...,"London, England, United Kingdom",0.698267


### Missing skills:
1. Experience with cloud computing platforms (e.g. AWS, Google Cloud Platform)
2. Containerization and orchestration (e.g. Docker, Kubernetes)
3. Web development (e.g. JavaScript, HTML, CSS)
4. Data visualization (e.g. Tableau, Power BI)
5. Database design and management
6. Agile development
7. Version control systems (e.g. Git)
8. Natural language processing (NLP)
9. Deep learning frameworks (e.g. TensorFlow, PyTorch)
10. Software engineering principles/best practices

In [11]:
missing_skills = job_market_assistant.get_missing_skills(
    query_job_postings, cv_3, "Azure Data Mining Engineer"
)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title Azure Data Mining Engineer

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.775601
1,Data Engineer,ecocareers,DATA ENGINEER LONDON HYBRID - 3 DAYS IN THE OF...,"London, England, United Kingdom",0.769516
2,Data Engineer,Adria Solutions Ltd,We are seeking a Data Engineer to join our fri...,"Manchester, England, United Kingdom",0.756505
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.745697
4,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.742178
5,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.740576
6,"Machine Learning Engineer - UK, Hybrid",Multiverse,We’re on a mission to provide equitable access...,"London, England, United Kingdom",0.738025


### Missing skills:
1. Experience with cloud computing platforms such as AWS or Google Cloud
2. Familiarity with database management systems like MySQL or MongoDB
3. Proficiency in HTML, CSS, and JavaScript
4. Knowledge of software development methodologies, such as Agile or Scrum
5. Experience with data visualization tools like Tableau or Power BI
6. Familiarity with DevOps tools and practices
7. Understanding of Natural Language Processing (NLP) concepts and techniques
8. Knowledge of front-end frameworks such as React or Angular
9. Experience with version control systems like Git
10. Understanding of data warehousing concepts and data modeling

## Benchmarks


### Experimenting with prompts


In [ ]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)


cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

### Model stability and execution time


#### ChatGPT


In [10]:
for _ in range(3):
    start_time = time.time()

    job_market_assistant = JobMarketAssistant(
        model=chat_gpt,
        cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
        job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
        key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
        missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    )

    cover_letter = job_market_assistant.get_cover_letter(
        file_job_postings.iloc[0], cv_1
    )

    missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1)

    display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

NameError: name 'cv_1' is not defined

#### Gemini


#### Llama
